# Imports

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")

# Date format

In [ ]:
df_train['Policy_Start_Date'] = pd.to_datetime(df_train['Policy_Start_Date'])
df_train['Policy_Start_Date_Int'] = (df_train['Policy_Start_Date']-pd.Timestamp("2019-08-17")) // pd.Timedelta('1D')

# przeksztalcam date, na liczbe dni ktore minely od pierwszego rejestru (2019-08-17)

# Columns

In [3]:
onehot_cols =['Gender','Education_Level', 'Smoking_Status', 'Property_Type']
onehot_null_cols = ['Marital_Status', 'Education_Level', 'Occupation', 'Location','Policy_Type' ]
num_null_cols = ['Age','Annual_Income', 'Number_of_Dependents', 'Health_Score', 'Vehicle_Age','Credit_Score' ]
ordinal_null_cols = ['Prevoius_Claims','Insurance_Duration', 'Customer_Feedback']
ordinal_cols = ['Policy_Start_Date_Int', 'Exercise_Frequency']